In [ ]:
import pandas as pd
from google.colab import files
from google.cloud import translate_v2 as translate
import os
import json
import time

In [ ]:
# Mengambil kredensial dari Colab Secrets
from google.colab import userdata
credentials_json = userdata.get('GCLOUD_CREDENTIALS')

In [ ]:
# Menyimpan credentials ke file sementara
with open('gcloud_credentials.json', 'w') as f:
    f.write(credentials_json)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gcloud_credentials.json'

In [ ]:
# Menginisialisasi client Translation API
translate_client = translate.Client()

In [ ]:
# Fungsi untuk menerjemahkan teks dengan retry
def translate_text(text, dest='id', retries=3):
    if pd.isna(text):
        return text
    for _ in range(retries):
        try:
            result = translate_client.translate(text, target_language=dest)
            return result['translatedText']
        except Exception as e:
            print(f"Error menerjemahkan: {e}. Mencoba lagi...")
            time.sleep(1)
    return text

In [ ]:
# Load dataset gejala
general_df = pd.read_csv("penyakitumum.csv")
digestive_df = pd.read_csv("penyakitpencernaan.csv")
skin_df = pd.read_csv("penyakitkulit.csv")
bone_df = pd.read_csv("penyakittulang.csv")

# Load dataset deskripsi dan pencegahan
desc_df = pd.read_csv("symptom_Description.csv")
prec_df = pd.read_csv("symptom_precaution.csv")

In [ ]:
# menggabungkan semua penyakit dari dataset gejala
all_diseases = set(general_df["Disease"].str.strip().str.lower()) | \
    set(digestive_df["Disease"].str.strip().str.lower()) | \
    set(skin_df["Disease"].str.strip().str.lower()) | \
    set(bone_df["Disease"].str.strip().str.lower())

# Penyakit dari dataset deskripsi dan pencegahan
desc_diseases = set(desc_df["Disease"].str.strip().str.lower())
prec_diseases = set(prec_df["Disease"].str.strip().str.lower())

In [ ]:
# Memvalidasi konsistensi untuk mencocokan penyakit
missing_in_desc = all_diseases - desc_diseases
missing_in_prec = all_diseases - prec_diseases
extra_in_desc = desc_diseases - all_diseases
extra_in_prec = prec_diseases - all_diseases

print("Penyakit di dataset gejala yang tidak ada di dataset deskripsi:", missing_in_desc)
print("Penyakit di dataset gejala yang tidak ada di dataset pencegahan:", missing_in_prec)
print("Penyakit di deskripsi yang tidak ada di dataset gejala:", extra_in_desc)
print("Penyakit di pencegahan yang tidak ada di dataset gejala:", extra_in_prec)

Penyakit di dataset gejala yang tidak ada di deskripsi: set()
Penyakit di dataset gejala yang tidak ada di pencegahan: set()
Penyakit di deskripsi yang tidak ada di gejala: {'hyperthyroidism', 'hepatitis d', 'diabetes', 'hypothyroidism', 'aids', '(vertigo) paroymsal  positional vertigo', 'acne', 'hepatitis b', 'hepatitis c', 'hepatitis a', 'hypoglycemia', 'hepatitis e', 'dimorphic hemorrhoids(piles)', 'alcoholic hepatitis', 'drug reaction', 'chronic cholestasis', 'jaundice', 'hypertension'}
Penyakit di pencegahan yang tidak ada di gejala: {'hyperthyroidism', 'hepatitis d', 'diabetes', 'hypothyroidism', 'aids', '(vertigo) paroymsal  positional vertigo', 'acne', 'dimorphic hemmorhoids(piles)', 'hepatitis b', 'hepatitis c', 'hepatitis a', 'hypoglycemia', 'hepatitis e', 'alcoholic hepatitis', 'drug reaction', 'chronic cholestasis', 'jaundice', 'hypertension'}


In [ ]:
# Jika ada penyakit yang hilang, memberi peringatan
if missing_in_desc or missing_in_prec:
    print("\nPeringatan: Beberapa penyakit di dataset gejala tidak ditemukan di dataset baru. "
          "Pastikan semua penyakit ada atau tambahkan secara manual.")

In [ ]:
# Menerjemahkan deskripsi
desc_df["Description"] = desc_df["Description"].apply(lambda x: translate_text(x))

# Menerjemahkan tindakan pencegahan
for col in ["Precaution_1", "Precaution_2", "Precaution_3", "Precaution_4"]:
    prec_df[col] = prec_df[col].apply(lambda x: translate_text(x))

# Menampilkan contoh isi kolom setelah penerjemahan
print("\nContoh isi symptom_description:")
print(desc_df.head())

print("\nContoh isi symptom_precaution:")
print(prec_df.head())


Contoh isi translated_symptom_description.csv:
          Disease                                        Description
0   Drug Reaction  Reaksi obat yang tidak diharapkan (ADR) adalah...
1         Malaria  Penyakit menular yang disebabkan oleh parasit ...
2         Allergy  Alergi adalah respons sistem imun terhadap zat...
3  Hypothyroidism  Hipotiroidisme, disebut juga tiroid kurang akt...
4       Psoriasis  Psoriasis adalah kelainan kulit umum yang memb...

Contoh isi translated_symptom_precaution.csv:
          Disease                           Precaution_1  \
0   Drug Reaction                       hentikan iritasi   
1         Malaria  Konsultasikan ke rumah sakit terdekat   
2         Allergy                       oleskan calamine   
3  Hypothyroidism                       mengurangi stres   
4       Psoriasis    cuci tangan dengan air sabun hangat   

                                     Precaution_2               Precaution_3  \
0           konsultasikan ke rumah sakit terdekat 

In [ ]:
# Menyimpan file terjemahan
desc_df.to_csv("symptom_description.csv", index=False)
prec_df.to_csv("symptom_precaution.csv", index=False)

In [ ]:
# Mengunduh file
files.download("symptom_description.csv")
files.download("symptom_precaution.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>